# Inverse Reachability Repositioning Debug

```sh
Pt: 
  x: 8.0
  y: 8.0
  theta: 0.0
Obs: 
  - 
    center: 
      x: 8.44674992463
      y: 7.98299052896
      theta: 1.68352296792
    size_x: 0.5
    size_y: 1.0
Cr: 
  x: -0.523598775598
  y: 1.57079632679
  z: 0.0
Ct: 
  x: -2.72136862653
  y: -2.65155545645
  z: 0.0
section_definition: 
  x: 0.05
  y: 1.0
  z: 0.02
preferred_heading: 0.0
```

## 1. Offline

### IRM Data

- Sample: ./config/robocare_right_irm.npy
  - This sample has 7899 points.
  - The supported z of target is {0.7, 0.75, 0.8}.

| Column Index | Name                     | Unit       | Remark           |
| ------------ | ------------------------ | ---------- | ---------------- |
| 0            | Mobile Base x            | meter      | For query output |
| 1            | Mobile Base y            | meter      | For query output |
| 2            | Target Object z (height) | meter      | For query input  |
| 3            | EEP x                    | meter      | For IK solver    |
| 4            | EEP y                    | meter      | For IK solver    |
| 5            | EEP z                    | meter      | For IK solver    |
| 6            | EE Roll                  | **DEGREE** |                  |
| 7            | EE Pitch                 | **DEGREE** |                  |
| 8            | EE Yaw                   | **DEGREE** | Cr               |
| 9            | Manipulability           | -          |                  |
| 10           | Joint_0 value            | radian     |                  |
| 11           | Joint_1 value            | radian     |                  |
| >=12         | Joint_2... values        | radian     |                  |


### Configuration

In [ ]:
base_radius = 0.3
irm_path = "../config/robocare_right_irm.npy"

### Initialization

In [2]:
%matplotlib widget

import numpy as np
import irm

reposition = irm.InverseReachabilityMap(irm_path, base_radius, is_jupyter=True)

In [3]:
print(reposition)

< Inverse Reachability Solver >
Configuration:
    Base radius: 0.300000 m
    IRM data: ../config/robocare_right_irm.npy
       Shape: (7899, 18)



## 2. Online

### 2.1. Input

Known (from offline)
1. `irm`: Inverse reachability map of the robot arm
2. `base_radius`: The radius of the robot base (base is assumed to be circular)

---

Input

1. `Pt`: Position of the target object (relative to the robot’s current pose) => `(x, y)`
2. `Obs`: Area list of ground obstacles => `[CollisionModel, ...]`
3. `Cr`: Constraints on the approach angle (relative to the robot heading) (-pi ~ pi) => `(min, max)`
4. `Ct`: Constraints on the approach angle (relative to the target heading) (-pi ~ pi) => `(min, max)`

In [4]:
from collision import CollisionBox

Pt = (8.0, 8.0)
Obs = [CollisionBox((8.4467, 7.9829), 1.6835, 0.5, 1.0)]
Cr = (-0.52359, 1.57079)  # min, max

Ct = (-2, 2)
# Ct = (-2.72136, -2.65155) # min, max

### 2.2. Process

1. Cut the range of `Cr` from `irm` and set it to `Fcut`.
2. ~~[SKIP] Scan the maximum points for the radius and angle by each circle in `Fcut`.~~
3. Extract only the maximum as a `Fmax`.
4. Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
5. Remove all obstacle areas from `Fwiped` with the offset of `base_radius`. => `Fclean`

In [5]:
try:
    reposition.calc(Pt, Obs, Cr, Ct, (0.05, 1.0, 0.02), verbose=True)
except:
    pass

######################
# 1. Fcut
######################
Cut the range of `Cr` from `Fraw` and set it to `Fcut`.
Lower constraints on Cr: %f deg -29.999497195254772
Upper constraints on Cr: %f deg 89.99963750135457
Input Cr range in degree:  [-90.0, -75.0, -60.0, -45.0, -30.0, -15.0, 0.0, 15.0, 30.0, 45.0, 60.0, 75.0, 90.0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

######################
# 2. Fmax
######################
And extract only the maximum as a `Fmax`.
sq_sections.shape:  (48,)
Fmax.shape:  (30, 18)
When Ct == 0.0 deg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

######################
# 3. Fwiped
######################
Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
min Ct:  -114.59155902616465
max Ct:  114.59155902616465
Fwiped.shape:  (4680, 20)
Interval is  0.01  [m].


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

######################
# 4. Fclean
######################
Remove all obstacle areas from `Fwiped` with the offset of `Rsize`. => `Fclean`
Fclean.shape:  (4680, 20)
Fclean.shape:  (2093, 20)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.3. Output

Candidate poses
(sorted in descending order of manipulability)

- `Cr`: Constraints on the approach angle (relative to the robot heading)
- `Ct`: Constraints on the approach angle (relative to the target heading)
- Chage of the robot heading `Theta = Ct - Cr`

| Column Index | Name                     | Unit       | Remark           |
| ------------ | ------------------------ | ---------- | ---------------- |
| 0            | Ct                       | **DEGREE** |                  |
| 1            | Cr                       | **DEGREE** | EE Yaw           |
| 2            | Mobile Base x            | meter      | For query output |
| 3            | Mobile Base y            | meter      | For query output |
| 4            | Manipulability           | -          |                  |
| 5            | Target Object z (height) | meter      | For query input  |
| 6            | EEP x                    | meter      | Based on base_footprint |
| 7            | EEP y                    | meter      | Based on base_footprint |
| 8            | EEP z                    | meter      | Based on base_footprint |
| 9            | EE Roll                  | **DEGREE** | Based on base_footprint |
| 10           | EE Pitch                 | **DEGREE** | Based on base_footprint |
| 11           | EE Yaw                   | **DEGREE** | Cr, Based on base_footprint |
| 12           | Joint_0 value            | radian     |                  |
| 13           | Joint_1 value            | radian     |                  |
| >=14         | Joint_2... values        | radian     |                  |


In [6]:
candidates = reposition.get_candidates(num=-1)
for c in candidates[:5]:
    reposition.candidate_inspector(c)

Candidate:
	   Base Pose2D: (x: 7.642 m, y: 7.980 m, theta: 3.203 deg)
	            Cr: 75.000 deg
	            Ct: 78.203 deg
	      Target Z: 0.700 m
	Manipulability: 0.10972
	  End-effecotr: (x: 0.331 m, y: -0.105 m, z: 0.700 m)
	                (r: -90.000 m, p: 90.000 m, y: 75.000 m) based on [base_footprint]
	   Joint (deg):  ['16.4', '-21.4', '-59.4', '82.6', '59.2', '59.6', '155.5', '-30.7']
Candidate:
	   Base Pose2D: (x: 7.642 m, y: 7.970 m, theta: 4.799 deg)
	            Cr: 75.000 deg
	            Ct: 79.799 deg
	      Target Z: 0.700 m
	Manipulability: 0.10972
	  End-effecotr: (x: 0.331 m, y: -0.105 m, z: 0.700 m)
	                (r: -90.000 m, p: 90.000 m, y: 75.000 m) based on [base_footprint]
	   Joint (deg):  ['16.4', '-21.4', '-59.4', '82.6', '59.2', '59.6', '155.5', '-30.7']
Candidate:
	   Base Pose2D: (x: 7.643 m, y: 7.960 m, theta: 6.395 deg)
	            Cr: 75.000 deg
	            Ct: 81.395 deg
	      Target Z: 0.700 m
	Manipulability: 0.10972
	  End-effecotr: 

## 3. Client-side

Service Output

1. `num_candidates` (uint8): The number of candidates
2. `candidates` (geometry_msgs/Pose2D[]): x, y, heading of the robot
3. `approach_angles` (float64[])
4. `manipulabilities` (float64[])
5. `joint_angles` (sensor_msgs/JointState[]): radian

```     
|                  [top view]         X
|                                     ^
|                                     |
|                              Y <---(O)  (base_footprint)
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|      [top view]                  [left view]            [backward view]
|    45     0    -45           90 <-----| (hand)            90(X, impossible)
|      .    ^    .                   .  |     ###             \|/
|        .  |  .               120 .   180    ###        0 ----+---- -180
| 90 <------|------> -90      ============| (ROBOT)            |
|                                 table   |                   -90
|        (1st Z)          =>           (Y)          =>      (2nd Z) = LWrist_Pitch - 90
|    approach angle              look-down angle          hand angle (Thumb direction)
```

---

### Candidates에서 Cost Function 사용

Parameters

1. `preferred_heading`

In [7]:
preferred_heading = np.radians(-60)


2 type of service

- input --> raw candidates
- input --> (raw candidates)+(cost func) --> one new position